In [19]:
# Load the usual suspects
import csv
import pandas as pd
import numpy as np
import os
import Toolbox as tb
import DataProcessFunctions as DP
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Initialize warning log container 
log = list()

In [3]:
# Load Pre-processed data from Data Processing PCA.ipynb
FM_data = pd.read_csv(os.path.dirname(os.getcwd()) + '\\FM_data.csv')
returns = pd.read_csv(os.path.dirname(os.getcwd()) + '\\returns_data.csv').set_index(["permno", "date"])
industry_code = pd.read_csv(os.path.dirname(os.getcwd()) + '\\industry_codes.csv').set_index(["permno", "date"])

tb.downcast(FM_data)
tb.downcast(returns)
tb.downcast(industry_code)

Before downcast: 2.942 GB and float64    102
int64        3
dtype: int64
After downcast: 1.460 GB and float32    102
int32        2
int8         1
dtype: int64
Before downcast: 0.043 GB and float64    1
dtype: int64
After downcast: 0.029 GB and float32    1
dtype: int64
Before downcast: 0.043 GB and float64    1
dtype: int64
After downcast: 0.029 GB and float32    1
dtype: int64


In [49]:
# Dates partitioning the training, validation, and test set according to Gu, Kelly, and Xiu (2020) by rolling-windows approach
tb.partition_dates(iteration = 0) # 30 splits --> iteration = [0;29]

(195701, 197501, 197501, 198701, 198701, 198801)

In [ ]:
# Compute training, validation, and test set
training, validation, test = DP.complete_data_process(industry_code, returns, FM_data, iteration = 0)

In [ ]:
# Kan lave nyt .py dokument med support function, som er de funktioner jeg bruger i de store funktioner i toolbox.py for at gøre det mere clean. Kan også overveje at lave toolbox2.py i stedet
# Jeg kan overveje at ændre pca funktionen, hvis jeg vil have flere linjer i min PCA figur, men det er ikke fokus lige nu 
# ryd op i kode i morgen lav mapper, toolbox2 osv. 